In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
import seaborn as sns
import datetime as dt
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pandasql import sqldf

In [22]:
df = pd.read_csv('data.csv')

In [23]:
df.columns

Index(['country', 'year', 'uniqueid', 'bank_account', 'location_type',
       'cellphone_access', 'household_size', 'age_of_respondent',
       'gender_of_respondent', 'relationship_with_head', 'marital_status',
       'education_level', 'job_type'],
      dtype='object')

In [27]:
x = df[['country', 'year', 'uniqueid', 'location_type',
       'cellphone_access', 'household_size', 'age_of_respondent',
       'gender_of_respondent', 'relationship_with_head', 'marital_status',
       'education_level', 'job_type']]


y = df['bank_account']

x_tr, x_test, y_tr, y_test = train_test_split(x, y,test_size=.25, random_state=12)

In [28]:
x_train, x_val, y_train, y_val = train_test_split(x_tr, y_tr, test_size=.20, random_state=11)

In [29]:
x_train['is_urban'] = np.where(x_train['location_type'] == 'Urban',1,0)
x_train['cellphone'] = np.where(x_train['cellphone_access'] == 'Yes',1,0)
x_train['is_male'] = np.where(x_train['gender_of_respondent'] == 'Male',1,0)

<ipython-input-29-fb20c62cd9e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['is_urban'] = np.where(x_train['location_type'] == 'Urban',1,0)
<ipython-input-29-fb20c62cd9e3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['cellphone'] = np.where(x_train['cellphone_access'] == 'Yes',1,0)
<ipython-input-29-fb20c62cd9e3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [30]:
x_train

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,is_urban,cellphone,is_male
2565,Kenya,2018,uniqueid_2566,Rural,Yes,4,48,Female,Child,Married/Living together,Primary education,Self employed,0,1,0
6930,Rwanda,2016,uniqueid_863,Rural,Yes,2,27,Female,Head of Household,Single/Never Married,Primary education,Informally employed,0,1,0
11849,Rwanda,2016,uniqueid_5782,Rural,Yes,6,18,Female,Child,Single/Never Married,Primary education,Informally employed,0,1,0
1983,Kenya,2018,uniqueid_1984,Rural,Yes,1,62,Female,Head of Household,Divorced/Seperated,Primary education,Farming and Fishing,0,1,0
12181,Rwanda,2016,uniqueid_6114,Rural,Yes,4,25,Female,Spouse,Married/Living together,Primary education,Farming and Fishing,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15109,Tanzania,2017,uniqueid_307,Urban,Yes,2,36,Male,Head of Household,Single/Never Married,Primary education,Self employed,1,1,1
18123,Tanzania,2017,uniqueid_3321,Urban,Yes,1,41,Female,Head of Household,Widowed,Primary education,Informally employed,1,1,0
22138,Uganda,2018,uniqueid_716,Rural,Yes,10,76,Male,Head of Household,Widowed,Secondary education,Self employed,0,1,1
21199,Tanzania,2017,uniqueid_6397,Rural,Yes,2,31,Male,Head of Household,Single/Never Married,Secondary education,Informally employed,0,1,1
